In [1]:
import warnings # default warn => SettingWithCopyWarning
warnings.simplefilter(action='ignore', category=UserWarning)
import pandas as pd; pd.options.mode.chained_assignment = None
import numpy as np
from datetime import datetime, date
import os, time
import pickle
import scipy.fftpack
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt
import arrow
from pprint import pprint as pp
#from tqdm import tqdm

import dmyplant2
from dmyplant2 import (
    cred, MyPlant, Engine,
    FSMOperator, startstopFSM, FSM_splot, FSM_splotBC, FSM_VLine, FSM_add_Notations, FSM_add_Alarms, FSM_add_Warnings,
    bokeh_show, dbokeh_chart, add_dbokeh_vlines, get_cycle_data2, disp_result, disp_alarms, disp_warnings,
    cvset, cplotdef, equal_adjust, count_columns, load_data, get_cycle_data, get_cycle_data2)

import ipywidgets as widgets
from ipywidgets import AppLayout, Button, Text, Select, Tab, Layout, VBox, HBox, Label, HTML, interact, interact_manual, interactive, IntSlider, Output

from IPython.display import HTML, display
display(HTML("<style>.container {width:94% !important;}</style>"))

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Label, Text, Span, HoverTool #, Range1d#, LabelSet
output_notebook(hide_banner=True)

# login to myplant()
cred()
mp = MyPlant(3600)
Engine._list_cached_validations();
#mp._fetch_installed_base(); # refresh local installed fleet database

In [2]:
#lookup = 'BMW Landshut 4.10'
#lookup = 'GRR'
#lookup = 'V441'
lookup = 'Forsa Hartmoor'
motor_num = 1

In [3]:
def sfun(x):
    #return all([ (lookup in str(x['Design Number'])),  (x['OperationalCondition'] != 'Decommissioned') ])
    return all([ (lookup in str(x['IB Site Name'])),  (x['OperationalCondition'] != 'Decommissioned') ])
fleet = mp.search_installed_fleet(sfun).drop('index', axis=1)
fleet = fleet.sort_values(by = "Engine ID",ascending=True).reset_index(drop='index')

In [4]:
motor = fleet.iloc[motor_num]
modes = ['undefined','OFF','MAN','AUTO']
success = [True,False]
#success = [True]
e=Engine.from_fleet(mp,motor)
pp_from=e['Commissioning Date']
#pp_from='2022-03-28 06:00'
#pp_to='2022-03-28 08:14'
pp_to=datetime.now()
motor['IB Site Name'] + ' ' + motor['Engine ID'], modes, success

('Forsa Hartmoor M02', ['undefined', 'OFF', 'MAN', 'AUTO'], [True, False])

In [5]:
# Run State Machines
fsm = FSMOperator(e, p_from=pp_from, p_to=pp_to)
fsm.run0(enforce=False, silent=True, debug=False)
fsm.run1(silent=True, successtime=300, debug=False) # run Finite State Machine
fsm.run2(silent = True, debug = True)

In [ ]:
rda = fsm.starts
rda['stime'] = rda.apply(lambda x: x['starttime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda['etime'] = rda.apply(lambda x: x['endtime'].strftime('%d.%m.%Y %H:%M:%S'), axis=1)
rda[['no','success','mode','stime','etime','startpreparation','starter','speedup','idle','synchronize','loadramp','cumstarttime','targetoperation','rampdown','coolrun','runout']].round(2)

In [ ]:
# PLotter
rdb = fsm.starts
vv = 123
def myfigures(e):
    return {
    'actors' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Ignition_ITPAvg'],'ylim': [-10, 30], 'color':'rgba(255,0,255,0.4)', 'unit':'°KW'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':['Various_Values_PosThrottle','Various_Values_PosTurboBypass'],'ylim': [-10, 110], 'color':['rgba(105,105,105,0.6)','rgba(165,42,42,0.4)'], 'unit':'%'},
    ],
    'lubrication' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['Hyd_PressCrankCase'],'ylim': [-100, 100], 'color':'orange', 'unit':'mbar'},
    {'col':['Hyd_PressOilDif'],'ylim': [0, 2], 'color':'black', 'unit': 'bar'},
    {'col':['Hyd_PressOil'],'ylim': [0, 10], 'color':'brown', 'unit': 'bar'},
    {'col':['Hyd_TempOil','Hyd_TempCoolWat','Hyd_TempWatRetCoolOut'],'ylim': [0, 110], 'color':['#2171b5','orangered','hotpink'], 'unit':'°C'},
    ],
    'exhaust' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Exhaust_TempCyl*'),'ylim': [400, 700], 'unit':'°C'},
    {'col':e.dataItemsCyl('Knock_Valve_Noise_Cyl*'),'ylim': [0, 4000], 'unit':'V'},
    ],
    'ignition' : [
    {'col':['Power_PowerAct'], 'ylim':(0,5000), 'color':'red', 'unit':'kW'},
    {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue', 'unit':'rpm'},
    {'col':['TecJet_Lambda1'],'ylim': [0, 3], 'color':'rgba(255,165,0,0.4)', 'unit':'-'},
    {'col':e.dataItemsCyl('Monic_VoltCyl*'),'ylim': [0, 100], 'unit':'kV'},
    {'col':e.dataItemsCyl('Ignition_ITPCyl*'),'ylim': [0, 40], 'unit':'°KW'},
    {'col':e.dataItemsCyl('Knock_KLS98_IntKnock_Cyl*'),'ylim': [-30, 60], 'unit':'%'},
    ],
        
}
lfigures = myfigures(fsm._e)
plotdef, vset = cplotdef(mp, lfigures)
#dmaxlength = 1800
dmaxlength = None
dminlength = None
startversuch = rdb.iloc[vv]
ftitle = f"{fsm._e} ----- Start {startversuch['no']} {startversuch['mode']} | {'SUCCESS' if startversuch['success'] else 'FAILED'} | {startversuch['starttime'].round('S')}"
data = get_cycle_data2(fsm, startversuch, max_length=dmaxlength, min_length=dminlength, cycletime=1, silent=True, p_data=vset)
data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
for doplot in plotdef[:1]:
    dset = lfigures[doplot]
    ltitle = f"{ftitle} | {doplot}"
    try:
        if count_columns(dset) > 12: # no legend, if too many lines.
            fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, legend=False, figsize=(18, 10))
        else:
            fig = FSM_splot(fsm, startversuch, data, dset, title=ltitle, figsize=(16, 10))

        fig = FSM_add_Notations(fig, fsm, startversuch)
        disp_alarms(startversuch)
        disp_warnings(startversuch)
        fig = FSM_add_Alarms(fig, fsm, startversuch)
        fig = FSM_add_Warnings(fig, fsm, startversuch)
        bokeh_show(fig)
    except Exception as err:
        print(err)

In [ ]:
from dowork import plot_plotly
fig2 = plot_plotly(fsm, data, startversuch, cvset(mp,dset), dset, dfigsize=(16,10))
fig2.show(config={'scrollZoom': True})

In [ ]:
from dowork import plot_plotly, plotly_verstehen
ddset = [{'col':['Power_SetPower','Power_PowerAct'], 'ylim':(0,5000), 'color':['dodgerblue','red'], 'unit':'kW'},
        {'col':['Hyd_TempOil'],'ylim': [0, 100], 'color':'violet', 'unit':'°C'},
        {'col':['Hyd_PressOil','Hyd_PressOilDif'],'ylim': [0, 10], 'color':['indigo','darkorange'], 'unit':'bar'},
        {'col':['Various_Values_SpeedAct'],'ylim': [0, 2500], 'color':'blue','unit':'rpm'}]
dvset = cvset(mp,ddset)
data = get_cycle_data(fsm, startversuch, max_length=None, min_length=None, cycletime=1, silent=True, p_data=dvset)
#data['power_diff'] = pd.Series(np.gradient(data['Power_PowerAct']))
fig2 = plot_plotly(fsm, data, startversuch,dvset, ddset, dfigsize=(16,10))
config = dict({'scrollZoom': True})
fig2.add_vline(x=startversuch['endtime'],line_dash='solid',line_width=1,line_color='red')
fig2.add_hline(y=1500.0,yref='y2',ysizemode='scaled',line_dash='solid',line_color='blue',line_width=3)
fig2.show(config=config)
#[e for rec in ddset for e in rec['col']]

In [ ]:
# from dowork import plotly_verstehen
plotly_verstehen(fsm, startversuch, data)

In [ ]:
svdetail=fsm.starts.iloc[6].to_dict()
#pp(svdetail);

In [ ]:
for i, v in enumerate(fsm.runlogdetail(startversuch, statechanges_only=True)):
    print(f"{i:3} {v}")